In [3]:
##############################################################################
#
#            Requirements:
#              - Python 3.5+
#              - Exiv2 installation
# 
##############################################################################

def read_config_values():
    """Reads values from the configuration files and returns dictionary"""
    
    if config_file_exists():
        import image_caption_config
    else:
        raise ValueError ('Missing file image_caption_config.py, please refer to documentation') 
    
    # Values in a hidden config file override the config file, useful to prevent user config settings being pushed to github
    if hidden_config_file_exists():
        import _image_caption_config
        if image_caption_config.config.keys() != _image_caption_config.config.keys():
            raise ValueError ('keys in config files dont match hidden config file') 
        else:
            config_values = _image_caption_config.config
    else:
        config_values = image_caption_config.config
    
    return (config_values)


def config_file_exists():
    """ Check if a configuration file exists"""
    return(os.path.isfile('image_caption_config.py'))


def hidden_config_file_exists():
    """ Check if hidden configuration file exists"""
    return(os.path.isfile('_image_caption_config.py'))


def in_dir_exists(in_dir):
    """Check if in_dir directory exists"""
    return(os.path.isdir(in_dir))
            
            
def dir_exists(dir):
    """Check if out_dir directory exists"""
    return(os.path.isdir(dir))


def create_dir_and_parents(dir):
    """Create out_dir"""
    pathlib.Path(dir).mkdir(parents=True)

    
def dir_is_empty(dir):
    """Returns true if dir is empty otherwise false"""
    return (len(os.listdir(dir)) == 0)


def get_input_image(image_file_path):
    """Returns Pillow image at image_file_path and rotates if required"""
    input_image = Image.open(input_filepath)
    # If orientation is stored via exif metadata then transpose image and remove the orientation metadata
    input_image = ImageOps.exif_transpose(input_image)
    return(input_image)


def get_image_title(image_file_path):
    """Returns the XMP description of image at image_file_path (XMP description shows as 'title' in Windows Explorer)"""
    result = subprocess.run(['exiv2', '-g', 'Xmp.dc.description', '-Pv', image_file_path], stdout=subprocess.PIPE)
    title = result.stdout.decode(encoding="utf-8").strip()
    # remove text similiar to this 'lang="<something>"' that occurs at start of the string
    title = re.sub('lang=".*" ', '', title)
    return (title)


def title_is_valid_for_caption(title, caption_prefix):
    """If returns true if title is not empty and starts with caption_prefix"""
    if title == '' or title is None:
        return (False)
    elif inputs['caption_prefix'] == '':
        return (True)
    elif title[:len(inputs['caption_prefix'])] == inputs['caption_prefix']:
        return (True)
    else:
        return (False)

def image_title_ex_prefix(title, caption_prefix):
    """Returns title excluding caption_prefix"""    
    return(title[-len(title) + len(inputs['caption_prefix']):].strip())


def generate_captioned_image(input_image, frame_ratio, border_ratio, border_colour, frame_colour, caption_font, font_ratio, font_colour, caption):
    """ Adds frame, border and text caption to Pillow image img"""
    
    original_image_width = input_image.size[0]
    original_image_height = input_image.size[1]
    max_original_image_side_length = max(original_image_width, original_image_height)
    border_thickness = round(max_original_image_side_length * border_ratio)
    frame_thickness =  round(max_original_image_side_length * frame_ratio)
    border_width = original_image_width + border_thickness * 2
    border_height = original_image_height + border_thickness * 2
    frame_width = border_width + frame_thickness * 2
    frame_height = border_height + frame_thickness * 2
    
    bordered_image = Image.new('RGB', (border_width, border_height), border_colour)
    bordered_image.paste(input_image, (border_thickness, border_thickness))
    framed_image = Image.new('RGB', (frame_width, frame_height), frame_colour)
    
    framed_image.paste(bordered_image, (frame_thickness, frame_thickness))
    
    return(framed_image)


# class CaptionedPhoto:
          
#     def _captioned_image(self):
#         """ Returns orginal image with added frame and boreder"""
#         bordered_image = self._border_image()
#         bordered_image.paste(self.original_image, (self.borders_width(), self.borders_width()))
#         framed_image = self._frame_image()
#         framed_image.paste(bordered_image, (self.frame_width(), self.frame_width()))
#         font_size = round(self._max_side_length() * inputs['font_ratio'])
#         caption_font = ImageFont.truetype(inputs['caption_font'], font_size)
#         x = framed_image.size[0]/2
#         y =framed_image.size[1] - self.frame_width() -self.borders_width() /2
#         ImageDraw.Draw(framed_image).text(xy=(x,y), text=self._title(), font=caption_font, fill =inputs['font_colour'], anchor='mm')
#         return (framed_image)
        
        
if __name__ == "__main__":
    
    from PIL import Image, ImageDraw, ImageFont, ImageOps
    import os, pathlib, subprocess, re
  
    # Read configuration inputs into a dictionary
    inputs = read_config_values()
    
    if not dir_exists(inputs['out_dir']):
        create_dir_and_parents(inputs['out_dir'])
    elif not dir_is_empty(inputs['out_dir']):
        raise ValueError ('Output directory (' +  inputs['out_dir'] +') is not empty') 
    
    for filename_and_extension in os.listdir(inputs['in_dir']):
        image_extensions_lower_case = ['.jpeg', '.jpg', '.png']
        base_filename, file_extension = os.path.splitext(filename_and_extension)        
        if file_extension.lower() in image_extensions_lower_case:
            input_filepath = os.path.join(inputs['in_dir'], base_filename + file_extension)
            print ('processing ' + input_filepath)
            input_image = get_input_image(input_filepath)
            image_title = get_image_title(input_filepath)
            if title_is_valid_for_caption(image_title, inputs['caption_prefix']):
                caption = image_title_ex_prefix(image_title, inputs['caption_prefix'])    
                output_image= generate_captioned_image(input_image=input_image, frame_ratio = inputs['frame_ratio'], 
                    border_ratio = inputs['border_ratio'], border_colour = inputs['border_colour'], 
                    frame_colour = inputs['frame_colour'],caption_font = inputs['caption_font'], 
                    font_ratio = inputs['font_ratio'], font_colour = inputs['font_colour'], caption = caption)                
            else:
                output_image = input_image
                        
            output_filepath = os.path.join(inputs['out_dir'], base_filename + file_extension)
            output_image.save(output_filepath)

    print('\nProcessing complete...')

processing /home/charl/OneDrive/Documents_Charl/Computer_Technical/Programming_GitHub/AddImageCaption/Test_In_Folder/20230408_182048.jpg
processing /home/charl/OneDrive/Documents_Charl/Computer_Technical/Programming_GitHub/AddImageCaption/Test_In_Folder/Landscape_Example.JPG
processing /home/charl/OneDrive/Documents_Charl/Computer_Technical/Programming_GitHub/AddImageCaption/Test_In_Folder/Portrait_Example_Captioned.jpg
processing /home/charl/OneDrive/Documents_Charl/Computer_Technical/Programming_GitHub/AddImageCaption/Test_In_Folder/Portrait_Example_Uncaptioned.jpg

Processing complete...


In [9]:
my_photo._original_image_height()

4032